<a href="https://colab.research.google.com/github/dk-wei/nlp-algo-implementation/blob/main/LSA_and_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before the state-of-the-art word embedding technique, Latent Semantic Analysis (LSA) and Latent Dirichlet Allocation (LDA) area good approaches to deal with NLP problems. Both LSA and LDA have same input which is **Bag of words in matrix format**. LSA focus on **reducing matrix dimension** while LDA solves **topic modeling** problems.

I will not go through mathematical detail and as there is lot of great material for that. You may check it from reference. For the sake of keeping it easy to understand, I did not do pre-processing such as stopwords removal. It is critical part when you use LSA, LSI and LDA. After reading this article, you will know:
 - Latent Semantic Analysis (LSA)
 - Latent Dirichlet Allocation (LDA)
 - Take Away


In [2]:
from sklearn.datasets import fetch_20newsgroups
train_raw = fetch_20newsgroups(subset='train')
test_raw = fetch_20newsgroups(subset='test')

x_train = train_raw.data
y_train = train_raw.target

x_test = test_raw.data
y_test = test_raw.target

# Latent Semantic Analysis (LSA)

LSA for natural language processing task was introduced by Jerome Bellegarda in 2005. The objective of LSA is reducing dimension for classification. The idea is that words will occurs in similar pieces of text if they have similar meaning. We usually use Latent Semantic Indexing (LSI) as an alternative name in NLP field.

First of all, we have m documents and n words as input. An m * n matrix can be constructed while column and row are document and word respectively. You can use count occurrence or TF-IDF score. However, TF-IDF is better than count occurrence in most of the time as high frequency do not account for better classification.

![](https://miro.medium.com/proxy/0*7r2GKRepjh5Fl41r.png)

The idea of TF-IDF is that high frequency may not able to provide much information gain. In another word, rare words contribute more weights to the model. Word importance will be increased if the number of occurrence within same document (i.e. training record). On the other hand, it will be decreased if it occurs in corpus (i.e. other training records). For detail, you may check this blog.

The challenge is that the matrix is very sparse (or high dimension) and noisy (or include lots of low frequency word). So truncated SVD is adopted to reduce dimension.

![](https://miro.medium.com/max/1400/1*Z0EUVs7QElEqRqXtqut_FQ.png)

**The idea of SVD is finding the most valuable information and using lower dimension t to represent same thing.**

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

def build_lsa(x_train, x_test, dim=50):
    tfidf_vec = TfidfVectorizer(use_idf=True, norm='l2')
    svd = TruncatedSVD(n_components=dim)
    
    transformed_x_train = tfidf_vec.fit_transform(x_train)
    transformed_x_test = tfidf_vec.transform(x_test)
    
    print('TF-IDF output shape:', transformed_x_train.shape)
    
    x_train_svd = svd.fit_transform(transformed_x_train)
    x_test_svd = svd.transform(transformed_x_test)
    
    print('LSA output shape:', x_train_svd.shape)
    
    explained_variance = svd.explained_variance_ratio_.sum()
    print("Sum of explained variance ratio: %d%%" % (int(explained_variance * 100)))
    
    return tfidf_vec, svd, x_train_svd, x_test_svd


tfidf_vec, svd, x_train_lda, x_test_lda = build_lsa(x_train, x_test)

TF-IDF output shape: (11314, 130107)
LSA output shape: (11314, 50)
Sum of explained variance ratio: 8%


In [12]:
x_train[:5]

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

In [15]:
x_train_lda[:2]

array([[ 0.24972705, -0.06942864, -0.01311425, -0.05103983, -0.00403584,
        -0.0069866 , -0.0281033 , -0.03696149, -0.00102237, -0.01865511,
         0.00171095,  0.0094233 , -0.09034356, -0.01437922, -0.05764015,
        -0.00149756, -0.05819083,  0.06553346, -0.01240388, -0.06645535,
         0.07101369,  0.01199455,  0.0445082 ,  0.03394705, -0.04561099,
        -0.05714162, -0.03763445,  0.02101698, -0.01736399,  0.01639008,
         0.04381269, -0.01933203, -0.00101538,  0.03542597, -0.02547384,
         0.00398878,  0.0591342 ,  0.01701092, -0.0163664 , -0.08605762,
         0.00710651, -0.05365106,  0.01285126, -0.04646758,  0.04073939,
         0.04262537,  0.0565492 ,  0.08231343,  0.01841167, -0.03029937],
       [ 0.1399918 , -0.07671644, -0.03975164, -0.02232068,  0.01438013,
         0.03615991, -0.03166721, -0.01104911, -0.03521134,  0.01150632,
        -0.02008854,  0.01800078,  0.03558538,  0.00249563, -0.02585091,
        -0.01435933,  0.00067577, -0.00648094, -0.

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold

lr_model = LogisticRegression(solver='newton-cg',n_jobs=-1)
lr_model.fit(x_train_lda, y_train)

cv = KFold(n_splits=5, shuffle=True)
    
scores = cross_val_score(lr_model, x_test_lda, y_test, cv=cv, scoring='accuracy')
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.6564 (+/- 0.0431)


# Latent Dirichlet Allocation (LDA)

LDA is introduced by David Blei, Andrew Ng and Michael O. Jordan in 2003. It is unsupervised learning and topic model is the typical example. The assumption is that each document mix with various topics and every topic mix with various words.
![](https://miro.medium.com/max/1138/1*SHahRtoGw3JP48e806DsIw.png)

Intuitively, you can image that we have two layer of aggregations. First layer is the distribution of categories. For example, we have finance news, weather news and political news. Second layer is distribution of words within the category. For instance, we can find “sunny” and “cloud” in weather news while “money” and “stock” exists in finance news.

However, “a”, “with” and “can” do not contribute on topic modeling problem. Those words exist among documents and will have roughly same probability between categories. Therefore, stopwords removal is a critical step to achieve a better result.

![](https://miro.medium.com/max/1270/1*qwA4jyRFBB6Htn3X4aftSw.png)

For particular document d, we get the topic distribution which is θ. From this distribution(θ), topic t will be chosen and selecting corresponding word from ϕ.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

def build_lda(x_train, num_of_topic=10):
    vec = CountVectorizer()
    transformed_x_train = vec.fit_transform(x_train)
    feature_names = vec.get_feature_names()

    lda = LatentDirichletAllocation(
        n_components=num_of_topic, max_iter=5, 
        learning_method='online', random_state=0)
    lda.fit(transformed_x_train)

    return lda, vec, feature_names

def display_word_distribution(model, feature_names, n_word):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        words = []
        for i in topic.argsort()[:-n_word - 1:-1]:
            words.append(feature_names[i])
        print(words)

lda_model, vec, feature_names = build_lda(x_train)
display_word_distribution(
    model=lda_model, feature_names=feature_names, 
    n_word=5)

Topic 0:
['the', 'for', 'and', 'to', 'edu']
Topic 1:
['c_', 'w7', 'hz', 'mv', 'ck']
Topic 2:
['space', 'nasa', 'cmu', 'science', 'edu']
Topic 3:
['the', 'to', 'of', 'for', 'and']
Topic 4:
['the', 'to', 'of', 'and', 'in']
Topic 5:
['the', 'of', 'and', 'in', 'were']
Topic 6:
['edu', 'team', 'he', 'game', '10']
Topic 7:
['ax', 'max', 'g9v', 'b8f', 'a86']
Topic 8:
['db', 'bike', 'ac', 'image', 'dod']
Topic 9:
['nec', 'mil', 'navy', 'sg', 'behanna']


# Take Away

- Both of them use **Bag-of-words** as input matrix
- The challenge of SVD is that we are **hard to determine the optimal number of dimension**. In general, low dimension consume less resource but we may not able to distinguish opposite meaning words while high dimension overcome it but consuming more resource.


##  附 Truncated SVD

In [22]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import random as sparse_random
X = sparse_random(100, 100, density=0.01, format='csr',
                  random_state=42)

X.shape

(100, 100)

In [23]:
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
svd.fit(X)

TruncatedSVD(algorithm='randomized', n_components=5, n_iter=7, random_state=42,
             tol=0.0)

In [24]:
print(svd.explained_variance_ratio_)

[0.0646051  0.06339479 0.06394407 0.05352903 0.04062679]


In [26]:
print(svd.explained_variance_ratio_.sum())

0.2860997781448586


In [27]:
print(svd.singular_values_)

[1.55360944 1.5121377  1.51052009 1.37056529 1.19917045]


来看看我们具体的输出值是怎样的

In [29]:
svd.fit_transform(X).shape

(100, 5)

In [30]:
svd.fit_transform(X)[:2]

array([[-5.07703085e-17, -1.98918547e-16,  5.27579738e-17,
        -3.73171500e-16,  5.45109431e-18],
       [-2.50932551e-06,  7.11886244e-03, -4.80647414e-06,
        -3.06570234e-05, -3.15999399e-04]])